# Data Validation and Sanity Checks

In this notebook, we'll compare the outputs of our GeneExpressionData class with the original expression matrices and labels

In [1]:
import pandas as pd 
import os
import sys
import pandas as pd
import numpy as np
from torch.utils.data import *
from tqdm import tqdm
import linecache 
from pytorch_tabnet.tab_model import TabNetClassifier
sys.path.append('../src/')
sys.path.append('..')

from src.models.lib.neural import GeneClassifier

In [2]:
from src.models.lib.data import *
from src.helper import *

## Intersection between reference columns with current dataset columns 

Here, we'll write test code for the methods used in mapping an arbitrary dataset sample to a list of given reference columns, since this method being correct is extremely important 

In [3]:
def clean_sample(sample, refgenes, currgenes):
    intersection = np.intersect1d(currgenes, refgenes, return_indices=True)
    indices = intersection[1] # List of indices in currgenes that equal refgenes 
    
    axis = (1 if sample.ndim == 2 else 0)
    sample = np.sort(sample, axis=axis)
    sample = np.take(sample, indices, axis=axis)

    return torch.from_numpy(sample)

### Unit Test Example 01

In [4]:
def test1():
    ref = ['a', 'b', 'c']
    curr = ['b', 'a', 'c', 'd'] 
    sample = np.array([1,2,3,4]) # Want --> [2,1,3]

    result = clean_sample(sample, ref, curr)
    desired = torch.from_numpy(np.array([2,1,3]))
    
    assert torch.equal(result, desired)
    
def test2():
    ref = ['a', 'b', 'c']
    curr = ['c', 'd', 'b', 'a']

    sample = np.array(
        [[1,2,3,4],
         [5,6,7,8]]
    ) 
    # --> want [[4, 3, 1],
    #           [8, 7, 5]]

    res = clean_sample(sample, ref, curr)
    desired = torch.from_numpy(np.array([
        [4,3,1],
        [8,7,5]
    ]))
    
    assert torch.equal(res, desired)
    
test1()
test2()

From initial tests, `clean_sample` seems to be working correctly.

## Validation of GeneExpressionData class with the original expression matrices and label files 

In this section, we'll confirm that the GeneExpressionData method returns the correct (sample, label) pairs relative to the original expression matrices and raw label files.

In [5]:
datafiles, labelfiles = list(INTERIM_DATA_AND_LABEL_FILES_LIST.keys()), list(INTERIM_DATA_AND_LABEL_FILES_LIST.values())

datafiles = [os.path.join('..', 'data', 'interim', f) for f in datafiles]
labelfiles = [os.path.join('..', 'data', 'processed/labels', f) for f in labelfiles]

for labelfile in labelfiles:
    display(pd.read_csv(labelfile)['cell'])

0              0
1              1
2              2
3              3
4              4
           ...  
186471    189404
186472    189405
186473    189406
186474    189407
186475    189408
Name: cell, Length: 186476, dtype: int64

0            1
1            2
2            3
3            4
4            5
         ...  
47504    49489
47505    49490
47506    49491
47507    49492
47508    49493
Name: cell, Length: 47509, dtype: int64

0            0
1            1
2            2
3            3
4            4
         ...  
76528    76528
76529    76529
76530    76530
76531    76531
76532    76532
Name: cell, Length: 76533, dtype: int64

0              0
1              1
2              2
3              3
4              4
           ...  
659546    691923
659547    691924
659548    691925
659549    691926
659550    691927
Name: cell, Length: 659551, dtype: int64

Next, we define a function that takes the first `N` samples from the GeneExpressionData object and from the raw expression matrix and compares the samples to make sure they are equal.

In [6]:
N = 5

def test_first_n(n, datafile, labelfile):
    data = GeneExpressionData(datafile, labelfile, 'Type', skip=3, index_col='cell')
    cols = data.columns
    
    # Generate dict with half precision values to read this into my 16gb memory
    data_df = pd.read_csv(datafile, nrows=2*n, header=1, dtype=np.float32) # Might need some extras since numerical index drops some values
    label_df = pd.read_csv(labelfile, nrows=n)
    similar = []
    for i in range(n):
        datasample = data[i][0]

        idx = label_df.loc[i, 'cell']
        dfsample = torch.from_numpy(data_df.loc[idx, :].values).float()
        
        isclose = all(torch.isclose(datasample, dfsample))
        
        similar.append(isclose)
    
    print(f"First {n=} columns of expression matrix is equal to GeneExpressionData: {all(p for p in similar)}")

def test_split(n, datafile, labelfile, class_label='Type', index_col='cell'):
    from sklearn.model_selection import train_test_split 
    # 2*n is usually enough to capture differences in index_col

    label_df = pd.read_csv(labelfile, nrows=n)
    data_df = pd.read_csv(datafile, nrows=2*n + 1, dtype=np.float32, header=1)
    display(label_df)

    labels = label_df.loc[:, class_label]
    
    train, _ = train_test_split(labels, random_state=42)
    train = train.index 
    
    train_labeldf = label_df.loc[train, :].reset_index(drop=True)

    train = GeneExpressionData(
            datafile,
            labelfile,
            index_col=index_col,
            class_label=class_label,
            indices=train,
        )
    
    train_similar = []
    for i in range(n):
        trainsample = train[i][0]
        df_idx = train_labeldf.loc[i, index_col]
        print(df_idx)
        trainsample_fromdf = torch.from_numpy(
            data_df.loc[df_idx, :].values
        )
        
        train_similar.append(
            all(torch.isclose(trainsample, trainsample_fromdf))
        )
        
    assert all(p for p in train_similar) # all train are similar 


In [7]:
# test_split(10, datafiles[1], labelfiles[1])

In [8]:
# train._labeldf

## h5ad data tests (via numpy array indexing)

In [9]:
import scanpy as sc 
import anndata as an 

rand = pd.DataFrame(np.random.randint(0, 10, size=[10, 10]))
obs = [f'cell{i}' for i in range(10)]
var = [f'col{i}' for i in range(10)]

In [10]:
labels = np.random.randint(0, 2, size=[10])
labels

array([1, 1, 1, 0, 1, 0, 1, 0, 0, 0])

In [11]:
labels = pd.Series(labels, obs)
labels.index.name = 'cell'

In [12]:
labels.to_csv('h5ad_test_labels.csv')

In [13]:
labels = pd.read_csv('h5ad_test_labels.csv')
labels.head()

,cell,0
0,cell0,1
1,cell1,1
2,cell2,1
3,cell3,0
4,cell4,1


In [14]:
dataset=an.AnnData(
    X=rand,
    obs=obs,
    var=var
)

In [15]:
np.linalg.norm(dataset.X - rand) # good, nothing is changed

0.0

In [16]:
from sklearn.model_selection import train_test_split

trainindex, valindex = train_test_split(labels.loc[:, '0'], random_state=0)

In [17]:
trainindex, valindex

(9    0
 1    1
 6    1
 7    0
 3    0
 0    1
 5    0
 Name: 0, dtype: int64,
 2    1
 8    0
 4    1
 Name: 0, dtype: int64)

In [18]:
train, val = dataset.X[trainindex.index], dataset.X[valindex.index]

In [19]:
from scipy.sparse import issparse 
from torch.utils.data import Dataset

class NumpyStream(Dataset):
    def __init__(self,
        matrix: np.ndarray,
        labels: List[any]=None,
        columns: List[any]=None,
        *args,
        **kwargs,
    ) -> None:
        super().__init__()
        self.data = matrix
        self.labels = labels 
        
    def __getitem__(self, idx):
        if isinstance(idx, slice):
            step = (1 if idx.step is None else idx.step)
            idxs = range(idx.start, idx.stop, step)
            return [self[i] for i in idxs]

        data = self.data[idx, :]
        return (
            torch.from_numpy(data), 
            self.labels[idx]
        )
    
    def __len__(self):
        return len(self.data)


In [20]:
testdata = NumpyStream(
    matrix=dataset.X[trainindex.index, :],
    labels=trainindex.values
)

In [21]:
testdata.data - train

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [22]:
dataset.X[trainindex.index, :]

array([[7., 2., 8., 2., 4., 1., 6., 5., 1., 8.],
       [2., 2., 6., 5., 7., 7., 6., 2., 3., 0.],
       [9., 3., 1., 4., 3., 8., 2., 1., 0., 6.],
       [9., 4., 5., 4., 1., 3., 2., 2., 8., 2.],
       [8., 6., 0., 3., 9., 4., 4., 0., 6., 5.],
       [6., 7., 9., 5., 2., 2., 9., 8., 2., 4.],
       [9., 6., 1., 2., 7., 7., 1., 4., 5., 5.]], dtype=float32)

In [23]:
rand.loc[9, :]

0    7
1    2
2    8
3    2
4    4
5    1
6    6
7    5
8    1
9    8
Name: 9, dtype: int64

## Comparison with already written method 



In [24]:
# Make stratified split on labels
current_labels = labels.loc[:, '0']

traintest, valtest = (
    NumpyStream(
        matrix=dataset.X[split.index],
        labels=split.values,
    )
    for split in [trainindex, valindex]
)

In [25]:
traintest.data - train

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [26]:
trainindex.values == traintest.labels

array([ True,  True,  True,  True,  True,  True,  True])

In [27]:
mouseatlas = an.read_h5ad('../data/mouse/MouseAdultInhibitoryNeurons.h5ad')

mouseatlas

AnnData object with n_obs × n_vars = 141069 × 34430
    obs: 'dataset_name', 'batch_name', 'timepoint', 'region', 'class', 'hires_leiden', 'leiden', 'n_genes', 'latent_cell_probability', 'latent_time', 'n_counts'
    var: 'feature_type-0-10', 'id-0-10', 'name-0-10', 'mean', 'std'
    uns: 'allen_class_label_colors', 'allen_cluster_label_colors', 'cc_velocity_graph', 'cc_velocity_graph_neg', 'cc_velocity_params', 'class_colors', 'classes', 'hvg', 'iroot', 'leiden', 'linear_velocity_graph', 'linear_velocity_graph_neg', 'linear_velocity_params', 'neighbors', 'pca', 'recover_dynamics', 'region_colors', 'regions', 'simplified_allen_colors', 'supervised_name_colors', 'umap', 'velocity_params'
    obsm: 'X_pca', 'X_umap', 'cc_velocity_umap', 'linear_velocity_umap'
    varm: 'PCs', 'loss'
    obsp: 'connectivities', 'distances'

In [36]:
mouselabels = pd.read_csv('../data/mouse/MouseAdultInhibitoryNeurons_labels.csv')
# mouselabels = mouselabels.loc[mouselabels['numeric_class'].isin(list(range(0, 5)))]

mouselabels

,class,numeric_class
0,S-phase_MCM4/H43C,36
1,S-phase_MCM4/H43C,36
2,Ctx_LHX6/SST,9
3,Str_LHX8/CHAT,40
4,Str_LHX8/CHAT,40
...,...,...
141064,S-phase_MCM4/H43C,36
141065,Transition,41
141066,Transition,41
141067,S-phase_MCM4/H43C,36


In [37]:
mouseatlas.X[0] - mouseatlas.X[141068]

array([0.      , 0.      , 2.229903, ..., 0.      , 0.      , 0.      ],
      dtype=float32)

In [40]:
mouseatlas.X[0] - mouseatlas.X[141065]

array([0.      , 0.      , 2.229903, ..., 0.      , 0.      , 0.      ],
      dtype=float32)

In [55]:
from xgboost import XGBClassifier

In [ ]:
classifier = XGBClassifier()

classifier.fit(mouseatlas.X, mouselabels.loc[:, 'numeric_class'].values)